In [1]:
from ctypes import *
from LadiskDAQ import dwfconstants as dwfc
import math
import time
import matplotlib.pyplot as plt
import sys
import numpy

%matplotlib qt


In [2]:
if sys.platform.startswith("win"):
    dwf = cdll.dwf
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
else:
    dwf = cdll.LoadLibrary("libdwf.so")

#declare ctype variables
hdwf = c_int()
sts = c_byte()
hzAcq = c_double(100000)
nSamples = 200000
rgdSamples = (c_double*nSamples)()
cAvailable = c_int()
cLost = c_int()
cCorrupted = c_int()
fLost = 0
fCorrupted = 0


print("Opening first device")
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == dwfc.hdwfNone.value:
    dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))

if hdwf.value == dwfc.hdwfNone.value:
    szerr = create_string_buffer(512)
    dwf.FDwfGetLastErrorMsg(szerr)
    print(str(szerr.value))
    print("failed to open device")
    quit()

print("Generating sine wave...")
dwf.FDwfAnalogOutNodeEnableSet(hdwf, c_int(0), dwfc.AnalogOutNodeCarrier, c_bool(True))
dwf.FDwfAnalogOutNodeFunctionSet(hdwf, c_int(0), dwfc.AnalogOutNodeCarrier, dwfc.funcSine)
dwf.FDwfAnalogOutNodeFrequencySet(hdwf, c_int(0), dwfc.AnalogOutNodeCarrier, c_double(1))
dwf.FDwfAnalogOutNodeAmplitudeSet(hdwf, c_int(0), dwfc.AnalogOutNodeCarrier, c_double(2))
dwf.FDwfAnalogOutNodeOffsetSet(hdwf, c_int(0), dwfc.AnalogOutNodeCarrier, c_double(2))
dwf.FDwfAnalogOutConfigure(hdwf, c_int(0), c_bool(True))

#set up acquisition
dwf.FDwfAnalogInChannelEnableSet(hdwf, c_int(0), c_bool(True))
dwf.FDwfAnalogInChannelRangeSet(hdwf, c_int(0), c_double(10))
dwf.FDwfAnalogInAcquisitionModeSet(hdwf, dwfc.acqmodeRecord)
dwf.FDwfAnalogInFrequencySet(hdwf, hzAcq)
dwf.FDwfAnalogInRecordLengthSet(hdwf, c_double(nSamples/hzAcq.value)) # -1 infinite record length

#wait at least 2 seconds for the offset to stabilize
time.sleep(2)

print("Starting oscilloscope")
dwf.FDwfAnalogInConfigure(hdwf, c_int(0), c_int(1))

cSamples = 0
acq_list = []
temp_samples = []

while cSamples < nSamples:
    dwf.FDwfAnalogInStatus(hdwf, c_int(1), byref(sts))
    if cSamples == 0 and (sts == dwfc.DwfStateConfig or sts == dwfc.DwfStatePrefill or sts == dwfc.DwfStateArmed) :
        # Acquisition not yet started.
        continue

    dwf.FDwfAnalogInStatusRecord(hdwf, byref(cAvailable), byref(cLost), byref(cCorrupted))
    
    cSamples += cLost.value

    if cLost.value :
        fLost = 1
    if cCorrupted.value :
        fCorrupted = 1

    if cAvailable.value==0 :
        continue

    if cSamples+cAvailable.value > nSamples :
        cAvailable = c_int(nSamples-cSamples)
    cSamples += cAvailable.value
    
    tempSamples = (c_double*cAvailable.value)()
    
    acq_list.append(cAvailable.value)
    #dwf.FDwfAnalogInStatusData(hdwf, c_int(0), byref(rgdSamples, sizeof(c_double)*cSamples), cAvailable) # get channel 1 data
    dwf.FDwfAnalogInStatusData(hdwf, c_int(0), byref(tempSamples), cAvailable) # get channel 1 data
    
    temp_samples.append( numpy.fromiter(tempSamples, dtype =float ))
    #dwf.FDwfAnalogInStatusData(hdwf, c_int(1), byref(rgdSamples, sizeof(c_double)*cSamples), cAvailable) # get channel 2 data


dwf.FDwfAnalogOutReset(hdwf, c_int(0))
dwf.FDwfDeviceCloseAll()

print("Recording done")
if fLost:
    print("Samples were lost! Reduce frequency")
if fCorrupted:
    print("Samples could be corrupted! Reduce frequency")


  
plt.plot(numpy.fromiter(rgdSamples, dtype =float))
plt.show()




Opening first device
Generating sine wave...
Starting oscilloscope
Recording done


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcsetup.all_backends

import pickle
import pyExSi

import LadiskDAQ
import os

%matplotlib qt

In [20]:

acq1 = LadiskDAQ.WaveFormsAcquisition(
    channels=[0, 1],    sample_rate=10000, 
    channel_names=None, acquisition_name="AD2 DEV", device_number=None
)
acq1.configure_channels(
    input_range={0: (-1, 1), 1: (-1, 1)},
)

In [25]:
subplot_options = {
            (0, 0): {
                'xlim': (0, 2),
                'ylim': (-1, 1),
                'axis_style': 'linear'
            },
            (1, 0): {
                'xlim': (0, 1000),
                'axis_style': 'linear'
            },
        }
layout = {
    "AD2 DEV":{
        (0, 0): [0, 1],
        (1, 0): [0, 1, "fft"],
    }
}

vis = LadiskDAQ.Visualization(layout=layout, subplot_options=subplot_options, 
                              nth=100, refresh_rate=100)


In [26]:
ldaq = LadiskDAQ.Core(acquisitions=[acq1], visualization=vis)

In [27]:
ldaq.run(5)

	Waiting for trigger...Measurement finished.


In [16]:
meas = ldaq.get_measurement_dict()["AD2 DEV"]
meas

{'data': array([[ 0.00435592, -0.0034559 ],
        [ 0.00435592, -0.0034559 ],
        [ 0.00435592, -0.0034559 ],
        ...,
        [ 0.00435592, -0.0034559 ],
        [ 0.00435592,  0.00024256],
        [ 0.00435592,  0.00024256]]),
 'time': array([0.000e+00, 1.000e-03, 2.000e-03, ..., 9.997e+00, 9.998e+00,
        9.999e+00]),
 'channel_names': ['CH0', 'CH1'],
 'sample_rate': 1000}

In [17]:
fig, axs = plt.subplots()
axs.plot(meas["time"], meas["data"][:, 0], label="Channel 0")
axs.plot(meas["time"], meas["data"][:, 1], label="Channel 1")